In [5]:
from enum import Enum
import numpy as np
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn import metrics
import math
import lime
import lime.lime_tabular
from sklearn.linear_model import Ridge
import random
from scipy.special import gammainccinv, erfinv
from sklearn.metrics import pairwise_distances
import pandas as pd
from matplotlib.patches import Ellipse
from scipy.stats import chi2, norm
from numpy.random import default_rng
from types import LambdaType


class Dataset(Enum):
  GENERATED = 'Generated'
  BLOBS = 'Blobs'
  IRIS2D = 'Iris (2D)'
  IRIS = 'Iris'
  BC = 'Breast cancer'
  PIMA = 'Pima Indian'
  WINE = 'Wine'

  @property
  def instance(self):
    if self == Dataset.GENERATED:
      amount = 2500
      data = np.random.uniform(0,[3,10],(amount,2))
      return Bunch(
        data=data,
        feature_names=['feature 1', 'feature 2'],
        target_names=['yellow', 'blue'],
        target=(np.logical_and(data[:,0]>1, data[:,1]>2)).astype(int)
      )

    if self == Dataset.BLOBS:
      from sklearn.datasets import make_blobs
      amount = 250
      X,y = make_blobs(amount, n_features = 3, centers = 4)
      return Bunch(
        data=X,
        feature_names=['feature 1', 'feature 2', 'feature 3'],
        target_names=['yellow', 'blue'],
        target=(y>1).astype(int)
      )

    if self == Dataset.IRIS2D:
      from sklearn.datasets import load_iris
      dataset = load_iris()
      indexes = np.array([0,1])
      dataset.data = dataset.data[:, indexes]
      dataset.feature_names = np.array(dataset.feature_names)[indexes]
      return dataset

    if self == Dataset.IRIS:
      from sklearn.datasets import load_iris
      return load_iris()

    if self == Dataset.BC:
      from sklearn.datasets import load_breast_cancer
      dataset = load_breast_cancer()
      return dataset

    if self == Dataset.WINE:
      from sklearn.datasets import load_wine
      dataset = load_wine()
      return dataset

    if self == Dataset.PIMA:
      from sklearn.datasets import load_diabetes
      dataset = load_diabetes()
      dataset.target = (dataset.target > 200).astype(int)
      return dataset


class Classifier(Enum):
  DT = 'Decision tree'
  RF = 'Random Forest'
  MLP = 'Perceptron'
  GNB = 'Naive Bayes'
  SVM = 'Support vector machine'
  QDA = 'Quadratic Discriminant Analysis'

  @property
  def instance(self):
    if self == Classifier.DT:
      from sklearn.tree import DecisionTreeClassifier
      return DecisionTreeClassifier(random_state=1)
    if self == Classifier.RF:
      from sklearn.ensemble import RandomForestClassifier
      return RandomForestClassifier(random_state=1, n_estimators=200)
    elif self == Classifier.MLP:
      from sklearn.neural_network import MLPClassifier
      return MLPClassifier(alpha=0.1, hidden_layer_sizes=(100,100,100))
    elif self == Classifier.GNB:
      from sklearn.naive_bayes import GaussianNB
      return GaussianNB()
    elif self == Classifier.SVM:
      from sklearn.svm import SVC
      return SVC(probability=True)
    elif self == Classifier.QDA:
      from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
      return QuadraticDiscriminantAnalysis()

In [6]:
from sklearn.preprocessing import normalize
from bisect import bisect
from scipy.optimize import newton
from functools import partial
from sklearn.linear_model import Ridge
from sklearn.utils import check_random_state
from sklearn.preprocessing import StandardScaler, MinMaxScaler


class LemonExplainer(object):
  """
  Intantiates the explainer.
  """
  def __init__(self, training_data, distance_kernel=None, sample_size = 5000, radius_max=1, random_state=None):
    self.random_state = check_random_state(random_state)
    np.random.seed(random_state)

    self.training_data = training_data
    self.scaler = StandardScaler(with_mean=False)
    self.scaler.fit(training_data)

    # Create hypersphere samples. The sphere is only computed once for performance and stability,
    # but it would be better to resample the sphere every time `explain_instance` is called.
    # I checked, this does not affect the results in any way.
    dimensions = training_data.shape[1]

    if distance_kernel is None:
      self.distance_kernel = np.vectorize(lambda x: x ** (1 / dimensions))
    else:
      self.distance_kernel = np.vectorize(self._transform(distance_kernel, dimensions, radius_max=radius_max))

    sphere = np.random.normal(size=(sample_size, dimensions))
    sphere = normalize(sphere)
    sphere *= self.distance_kernel(np.random.uniform(size=sample_size)).reshape(-1,1)

    self.sphere = sphere

  @property
  def surrogate(self):
    try:
      return self._surrgate
    except AttributeError:
      self._surrogate = Ridge(alpha=0, fit_intercept=True, normalize=True, random_state=self.random_state)
      return self._surrogate

  def explain_instance(self, instance, predict_fn, labels=(1,), surrogate=None):
    surrogate = surrogate or self.surrogate

    # Create transfer dataset by perturbing the original instance with the hypersphere samples
    X_transfer = self.scaler.inverse_transform(self.sphere) + np.array([instance])
    y_transfer = predict_fn(X_transfer)

    def explain_label(label):
      surrogate.fit(X_transfer, y_transfer[:,label])
      score = surrogate.score(X_transfer, y_transfer[:,label])
      return (surrogate.coef_, score)

    return [explain_label(label) for label in labels]

  def _transform(self, kernel, dimensions, sample_size = 1000, radius_max=1):
    """
    Inverse transform sampling
    """
    cdf_samples = np.array([kernel(x)*(x**(dimensions-1)) for x in np.linspace(0, radius_max, sample_size)])
    cdf_samples = np.cumsum(cdf_samples)
    cdf_samples /= cdf_samples[-1]
    return lambda y: radius_max * (bisect(cdf_samples, y) / sample_size)


def uniform_kernel(x):
    return 1


def gaussian_kernel(x, kernel_width):
    # https://github.com/marcotcr/lime/blob/fd7eb2e6f760619c29fca0187c07b82157601b32/lime/lime_tabular.py#L251
    return np.sqrt(np.exp(-(x ** 2) / kernel_width ** 2))


def sqcos_kernel(x):
    return np.cos(x)**2


def trapezoid_kernel(x, a, b):
    if 0 <= x and  x <= a:
        return (2 / (a + b))
    elif a <= x and x <= b:
        return (2 / (a + b)) * ((b - x) / (b - a))
    else:
        return 0

In [7]:
# For the `plot_sample_data` debug feature in the next cell, you need to patch LIME as such:

# Change return statement from `explain_instance_with_data` (lime_base.py:204) to

#         return (easy_model.intercept_,
#                 sorted(zip(used_features, easy_model.coef_),
#                        key=lambda x: np.abs(x[1]), reverse=True),
#                 prediction_score, local_pred, neighborhood_data[:, used_features], weights)

# And `ret_exp` in `explain_instance` (lime_tabular.py:450) to

#             (ret_exp.intercept[label],
#              ret_exp.local_exp[label],
#              ret_exp.score, ret_exp.local_pred, ret_exp.data, ret_exp.weights) = self.base.explain_instance_with_data(
#                     scaled_data,
#                     yss,
#                     distances,
#                     label,
#                     num_features,
#                     model_regressor=model_regressor,
#                     feature_selection=self.feature_selection)

#print sklearn.__version__
import sklearn


print("sklearn version:", sklearn.__version__)

sklearn version: 1.1.3


In [8]:

# MARS surrogate
# from pyearth import Earth
from pwla_lime.pwla_line_3d import MultiRegionLimeTabularExplainer3D
def do_experiment(Xtr, classifier, KERNEL_SIZE, plot_sample_data=False):
  from scipy.special import gammainccinv, erfinv
  sample_size_lime = 5000
  sample_size_lemon = 5000

  DIMENSIONS = Xtr.shape[1]
  p = 0.999

  if type(KERNEL_SIZE) is LambdaType:
    KERNEL_SIZE = KERNEL_SIZE(DIMENSIONS)

  kernel_width_lime = KERNEL_SIZE
  radius = KERNEL_SIZE * np.sqrt(2*gammainccinv(DIMENSIONS/2, (1-p)))
  kernel_width_lemon = KERNEL_SIZE
  kernel_width_pwla = KERNEL_SIZE
  diameter = radius * 2

  # explainers
  surrogate_lime = Ridge(fit_intercept=True, random_state=123)
  # surrogate_pwla = Earth(max_degree=1)          # piecewise-linear

  explainer_lime = lime.lime_tabular.LimeTabularExplainer(
      Xtr,
      kernel_width=kernel_width_lime,
      feature_selection='none',
      sample_around_instance=True, # Important! Essentially: True = synthetic, False = observation-based sampling
      discretize_continuous=False
  )

  surrogate_lemon = Ridge(fit_intercept=True, random_state=123)
  # surrogate_pwla = Ridge(fit_intercept=True, random_state=123)

  # decision tree of maximum depth 3
  from sklearn.tree import DecisionTreeRegressor
  # from sklearn.tree import DecisionTreeClassifier

  # add to surrogate_lemon
  surrogate_pwla = DecisionTreeRegressor(max_depth=3, random_state=123)
  # plot decision tree after training
  # sdsd

  # # Initialize multi-region LIME explainer
  pwla_explainer = MultiRegionLimeTabularExplainer3D(
      Xtr,
      kernel_width=kernel_width_pwla,
      feature_selection='none',
      sample_around_instance=True, # Important! Essentially: True = synthetic, False = observation-based sampling
      discretize_continuous=True,
      n_regions=50,
  )




  explainer_lemon = LemonExplainer(
      Xtr,
      sample_size=sample_size_lemon,
      distance_kernel=partial(gaussian_kernel, kernel_width=kernel_width_lemon),
      radius_max=radius
  )

  Xeval = Xtr
  yeval = classifier.predict_proba(Xeval)
  ceval = classifier.predict(Xeval)
  scaled_data = explainer_lime.scaler.transform(Xeval)

  points = []
  points2 = []

  # for many test instances
  for i in range(0, Xeval.shape[0]):
      testInstance = (Xeval[i,:], classifier.predict(Xeval[i,:].reshape(1, -1))[0])
      print("Test instance:", i, testInstance)

      explanation_lime = explainer_lime.explain_instance(
          testInstance[0],
          classifier.predict_proba,
          num_samples=sample_size_lime,
          labels=(testInstance[1],),
          model_regressor=surrogate_lime
      )


      #     # Generate explanation
      pwla_explanation = pwla_explainer.explain_instance(
          testInstance[0], classifier.predict_proba, num_features=DIMENSIONS, num_samples=sample_size_lime, labels=(testInstance[1],),model_regressor=surrogate_pwla
      )

      explanation_lemon = explainer_lemon.explain_instance(
          testInstance[0],
          classifier.predict_proba,
          labels=(testInstance[1],),
          surrogate=surrogate_lemon
      )

      ## Misleading: self-reported scores from LIME and LEMON
      # lime_fidelity = explanation_lime.score
      # lemon_fidelity = explanation_lemon[0][1]

      ## Generate 50000 new samples in the neighborhood to compute R^2 between reference and surrogate models.
      sample_size = 50000
      dimensions = DIMENSIONS
      sphere = np.random.normal(size=(sample_size, dimensions))
      sphere = normalize(sphere)
      sphere *= explainer_lemon.distance_kernel(np.random.uniform(size=sample_size)).reshape(-1,1)
      X_transfer = explainer_lemon.scaler.inverse_transform(sphere) + np.array([testInstance[0]])
      y_transfer = classifier.predict_proba(X_transfer)
      scaled_lime = (X_transfer - explainer_lime.scaler.mean_) / explainer_lime.scaler.scale_
      scaled_pwla = (X_transfer - pwla_explainer.scaler.mean_) / pwla_explainer.scaler.scale_


      from sklearn.metrics import mean_squared_error
      pwla_fidelity = mean_squared_error(
        y_transfer[:,testInstance[1]],
        surrogate_pwla.predict(scaled_pwla), # X_transfer
        squared=False
      )

      lime_fidelity = mean_squared_error(
        y_transfer[:,testInstance[1]],
        surrogate_lime.predict(scaled_lime),
        squared=False
      )

      points.append([i, pwla_fidelity])
      points2.append([i, lime_fidelity])

  return np.array(points), np.array(points2)


## Sanity check (This requires patching LIME! See previous cell.)
# data = Dataset.IRIS.instance
# default_rng(seed=2).shuffle(data.data)
# Xtr, Xte, ytr, yte = train_test_split(data.data, data.target, test_size=0.6, random_state=123)
# classifier = Classifier.GNB.instance
# classifier.fit(Xtr, ytr)
# do_experiment(Xtr, classifier, 0.2, plot_sample_data=True)

data = Dataset.BC.instance
default_rng(seed=2).shuffle(data.data)
Xtr, Xte, ytr, yte = train_test_split(data.data, data.target, test_size=0.6, random_state=123)
classifier = Classifier.MLP.instance
classifier.fit(Xtr, ytr)
print(Xtr.shape[0])
points, points2 = do_experiment(Xtr, classifier, lambda n: np.sqrt(n) * .75, plot_sample_data=False)
score_pwla = np.mean(points[:,1])
score_lime = np.mean(points2[:,1])
print("PWLA score:", score_pwla)
print("LIME score:", score_lime)

227
Test instance: 0 (array([1.476e+01, 1.474e+01, 9.487e+01, 6.687e+02, 8.875e-02, 7.780e-02,
       4.608e-02, 3.528e-02, 1.521e-01, 5.912e-02, 3.428e-01, 3.981e-01,
       2.537e+00, 2.906e+01, 4.732e-03, 1.506e-02, 1.855e-02, 1.067e-02,
       2.163e-02, 2.783e-03, 1.727e+01, 1.793e+01, 1.142e+02, 8.808e+02,
       1.220e-01, 2.009e-01, 2.151e-01, 1.251e-01, 3.109e-01, 8.187e-02]), 1)
Number of points in each region: 100
Number of unique regions: 50
Region 0 score: 0.20860042247066068
Number of points in each region: 100
Number of unique regions: 50
Region 1 score: 0.31006966071193376
Number of points in each region: 100
Number of unique regions: 50
Region 2 score: 0.31038531328147656
Number of points in each region: 100
Number of unique regions: 50
Region 3 score: 0.2550223118694408
Number of points in each region: 100
Number of unique regions: 50
Region 4 score: 0.3230565221408108
Number of points in each region: 100
Number of unique regions: 50
Region 5 score: 0.2957539527049529

In [9]:
# Single-core version

# from tqdm.notebook import tqdm

# datasets = [
# #  Dataset.IRIS2D,
#   Dataset.WINE,
#   Dataset.PIMA,
#   Dataset.BC
# ]

# classifiers = [
#   Classifier.GNB,
#   Classifier.MLP,
#   Classifier.RF
# ]

# kernel_widths = [
#   0.1,
#   0.2,
#   0.3,
#   # 0.4,
#   0.5,
#   # 0.6,
#   # 0.7,
#   # 0.8,
#   # 0.9,
#   1.0,
#   # 1.5,
#   2.0,
#   # 2.5,
#   # 3.0,
#   # 3.5,
#   4.0
# ]

# columns = pd.MultiIndex.from_product([datasets, classifiers])
# results_lime = pd.DataFrame(0, columns=columns, index=kernel_widths)
# results_lemon = pd.DataFrame(0, columns=columns, index=kernel_widths)

# for dataset in tqdm(datasets, position=0, desc="datasets", leave=False):
#   for classifier in tqdm(classifiers, position=1, desc="classifiers", leave=False):
#     data = dataset.instance
#     Xtr = data.data
#     clf = classifier.instance
#     clf.fit(Xtr, data.target)

#     for kernel_width in tqdm(kernel_widths, position=2, desc="kernel_widths", leave=False):
#       #print(dataset, classifier, kernel_width)
#       points, points2 = do_experiment(Xtr, clf, kernel_width, plot_sample_data=False)
#       score_lemon = np.mean(points[:,1])
#       score_lime = np.mean(points2[:,1])
#       #print("Mean R^2 LEMON: ", score_lemon)
#       #print("Mean R^2 LIME: ", score_lime)
#       results_lime.loc[kernel_width, (dataset, classifier)] = score_lime
#       results_lemon.loc[kernel_width, (dataset, classifier)] = score_lemon

In [11]:
# Multi-core version

from tqdm.notebook import tqdm

datasets = [
#  Dataset.IRIS2D,
  Dataset.WINE,
  Dataset.PIMA,
  Dataset.BC
]

classifiers = [
  Classifier.GNB,
  Classifier.MLP,
  Classifier.RF
]

kernel_widths = [
  # 0.1,
  # 0.2,
  0.3,
  # 0.4,
  0.5,
  # 0.6,
  # 0.7,
  # 0.8,
  # 0.9,
  1.0,
  # 1.5,
  # 2.0,
  # 2.5,
  # 3.0,
  # 3.5,
  lambda n: np.sqrt(n) * .75,
  4.0
]

columns = pd.MultiIndex.from_product([datasets, classifiers])

def faithfulness(options):
  dataset, classifier, kernel_width = options
  data = dataset.instance
  Xtr = data.data
  clf = classifier.instance
  clf.fit(Xtr, data.target)

  points, points2 = do_experiment(Xtr, clf, kernel_width, plot_sample_data=False)
  score_lemon = np.mean(points[:,1])
  score_lime = np.mean(points2[:,1])
  return score_lime, score_lemon



import contextlib
import joblib
from tqdm.notebook import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()


from itertools import product
from joblib import delayed, Parallel, cpu_count

parameters = list(product(datasets, classifiers, kernel_widths))


with tqdm_joblib(tqdm(total=len(parameters))) as progress_bar:
  result = Parallel(n_jobs=cpu_count())(delayed(faithfulness)(p) for p in parameters)
  result

  0%|          | 0/45 [00:00<?, ?it/s]

In [12]:
result

[(0.043801732877815455, 0.425848674345467),
 (0.10315409966978478, 0.45064159723057323),
 (0.257842276906764, 0.43832799105266185),
 (0.6519841782984984, 0.5610117525941364),
 (0.8456387608264351, 0.5412501671123978),
 (0.10793459332019245, 0.3434296719366382),
 (0.30694289263843055, 0.41723764418812315),
 (0.22586756519995968, 0.41333642014028477),
 (0.5341772701272172, 0.5277707328777188),
 (0.8522570117168771, 0.5595128356072674),
 (0.11798957754026451, 0.33905032379356936),
 (0.18590597862737884, 0.3060477330193445),
 (0.1560573756847847, 0.29561305941192584),
 (0.37591284032814537, 0.3302404761656581),
 (0.5446898419906286, 0.3398484389152248),
 (0.05708735888353683, 0.32954562296005296),
 (0.07963985160757431, 0.38175859209547874),
 (0.11982872313290269, 0.4300336208093152),
 (0.38724401956831406, 0.481579924886472),
 (0.6870842249325237, 0.5147179798538724),
 (0.04980763230444553, 0.2339549452604763),
 (0.06955166324049634, 0.2849728600040605),
 (0.06876680636429891, 0.333546221

In [13]:
kernel_widths_cleaned = ['0.75*sqrt(n)' if type(k) is LambdaType else k for k in kernel_widths]
results_lime = pd.DataFrame(0, columns=columns, index=kernel_widths_cleaned)
results_pwla = pd.DataFrame(0, columns=columns, index=kernel_widths_cleaned)
for i, (dataset, classifier, kernel_width) in enumerate(parameters):
  res = result[i]
  if res is None:
      # either skip, or fill with NaN
      continue
  # score_lime, score_lemon = res
  score_lime = result[i][0]
  score_pwla = result[i][1]
  results_lime.loc['0.75*sqrt(n)' if type(kernel_width) is LambdaType else kernel_width, (dataset, classifier)] = score_lime
  results_pwla.loc['0.75*sqrt(n)' if type(kernel_width) is LambdaType else kernel_width, (dataset, classifier)] = score_pwla

C:\Users\Acer\AppData\Local\Temp\ipykernel_48476\3211460134.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.043801732877815455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results_lime.loc['0.75*sqrt(n)' if type(kernel_width) is LambdaType else kernel_width, (dataset, classifier)] = score_lime
C:\Users\Acer\AppData\Local\Temp\ipykernel_48476\3211460134.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.425848674345467' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results_pwla.loc['0.75*sqrt(n)' if type(kernel_width) is LambdaType else kernel_width, (dataset, classifier)] = score_pwla
C:\Users\Acer\AppData\Local\Temp\ipykernel_48476\3211460134.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will 

In [14]:
results_lime

Dataset.WINE                                Dataset.PIMA  \
             Classifier.GNB Classifier.MLP Classifier.RF Classifier.GNB   
0.3                0.043802       0.107935      0.117990       0.057087   
0.5                0.103154       0.306943      0.185906       0.079640   
1.0                0.257842       0.225868      0.156057       0.119829   
0.75*sqrt(n)       0.651984       0.534177      0.375913       0.387244   
4.0                0.845639       0.852257      0.544690       0.687084   

                                              Dataset.BC                 \
             Classifier.MLP Classifier.RF Classifier.GNB Classifier.MLP   
0.3                0.049808      0.141518       0.051841       0.423461   
0.5                0.069552      0.111235       0.150540       0.563640   
1.0                0.068767      0.104586       0.490640       0.604056   
0.75*sqrt(n)       0.253551      0.246699       0.512204       1.290813   
4.0                0.432399      0.418977       0.505216       0.351534   

                            
             Classifier.RF  
0.3               0.102725  
0.5               0.170862  
1.0               0.264872  
0.75*sqrt(n)      0.366529  
4.0               0.357598

In [15]:
results_pwla

Dataset.WINE                                Dataset.PIMA  \
             Classifier.GNB Classifier.MLP Classifier.RF Classifier.GNB   
0.3                0.425849       0.343430      0.339050       0.329546   
0.5                0.450642       0.417238      0.306048       0.381759   
1.0                0.438328       0.413336      0.295613       0.430034   
0.75*sqrt(n)       0.561012       0.527771      0.330240       0.481580   
4.0                0.541250       0.559513      0.339848       0.514718   

                                              Dataset.BC                 \
             Classifier.MLP Classifier.RF Classifier.GNB Classifier.MLP   
0.3                0.233955      0.235632       0.424340       0.528349   
0.5                0.284973      0.251092       0.456885       0.509978   
1.0                0.333546      0.247987       0.430665       0.477716   
0.75*sqrt(n)       0.405599      0.240615       0.355586       0.600839   
4.0                0.430719      0.239802       0.363668       0.439647   

                            
             Classifier.RF  
0.3               0.292912  
0.5               0.274307  
1.0               0.205905  
0.75*sqrt(n)      0.169580  
4.0               0.169787

In [16]:
np.mean(np.mean((results_lime - results_pwla) / results_lime))

-1.1732518466273494